# Creating supercells with pymatgen

The [Pymatgen](https://pymatgen.org/) python library allows to setup
solid-state calculations using a flexible set of classes as well as an API
to an online data base of structures. Its `Structure` and `Lattice`
objects are directly supported by the DFTK `load_atoms` and `load_lattice`
functions, such that DFTK may be readily used to run calculation on systems
defined in pymatgen. Using the `pymatgen_structure` function a conversion
from DFTK to pymatgen structures is also possible. In the following we
use this to create a silicon supercell and find its LDA ground state
using direct minimisation.

First we setup the silicon lattice in DFTK.

In [1]:
using DFTK

a = 10.263141334305942  # Lattice constant in Bohr
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms = [Si => [ones(3)/8, -ones(3)/8]];

Next we make a `[2, 2, 2]` supercell using pymatgen

In [2]:
pystruct = pymatgen_structure(lattice, atoms)
pystruct.make_supercell([2, 2, 2])
lattice = load_lattice(pystruct)
atoms = [Si => [s.frac_coords for s in pystruct.sites]];

Setup an LDA model and discretize using
a single kpoint and a small `Ecut` of 5 Hartree.

In [3]:
model = model_LDA(lattice, atoms)
basis = PlaneWaveBasis(model, 5, kgrid=(1, 1, 1))

PlaneWaveBasis (Ecut=5.0, 1 kpoints)

Find the ground state using direct minimisation (always using SCF is boring ...)

In [4]:
scfres = direct_minimization(basis, tol=1e-5);

Iter     Function value   Gradient norm 
     0     1.123030e+02     1.486757e+00
 * time: 0.09415197372436523
     1     1.179682e+01     9.216697e-01
 * time: 1.4299280643463135
     2    -1.132875e+01     1.027030e+00
 * time: 1.5699858665466309
     3    -3.360850e+01     7.421950e-01
 * time: 1.7445080280303955
     4    -4.707207e+01     5.592409e-01
 * time: 1.9003610610961914
     5    -5.670812e+01     2.325899e-01
 * time: 2.053025960922241
     6    -5.971467e+01     1.550981e-01
 * time: 2.1631009578704834
     7    -6.090929e+01     6.907517e-02
 * time: 2.282309055328369
     8    -6.136620e+01     5.176941e-02
 * time: 2.4007949829101562
     9    -6.162082e+01     5.266173e-02
 * time: 2.511492967605591
    10    -6.182884e+01     2.734995e-02
 * time: 2.630743980407715
    11    -6.196884e+01     2.164420e-02
 * time: 2.7533750534057617
    12    -6.203799e+01     1.718940e-02
 * time: 2.8683559894561768
    13    -6.208535e+01     1.419336e-02
 * time: 2.9884459972381

In [5]:
scfres.energies

Energy breakdown:
    Kinetic             25.7671085
    AtomicLocal         -18.8557654
    AtomicNonlocal      14.8522599
    Ewald               -67.1831486
    PspCorrection       -2.3569765
    Hartree             4.8485382 
    Xc                  -19.3336826

    total               -62.261666454545
